## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# Drop the 'EIN' and 'NAME' columns
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Verify the columns have been dropped
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column
unique_values = application_df.nunique()

# Display the number of unique values
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Get the value counts of APPLICATION_TYPE
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Define a threshold for what you consider to be "less common"
threshold = 100  # You can adjust this threshold based on your data

# Replace less common application types with 'Other'
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(
    lambda x: 'Other' if application_type_counts[x] < threshold else x
)

# Verify the changes
print(application_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64


In [ ]:
# Handle CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()
cutoff_value = 100  # Adjust this value based on your data
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'CLASSIFICATION']  # Add other columns as needed
application_df = pd.get_dummies(application_df, columns=categorical_columns)

# Display the first few rows of the updated DataFrame
application_df.head()


,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,...,CLASSIFICATION_C1270,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2700,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_Other
0,Independent,ProductDev,Association,1,0,N,5000,1,False,True,...,False,False,False,False,False,False,False,False,False,False
1,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,False,False,...,False,False,True,False,False,False,False,False,False,False
2,CompanySponsored,ProductDev,Association,1,0,N,5000,0,False,False,...,False,False,False,False,False,True,False,False,False,False
3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,False,False,...,False,False,True,False,False,False,False,False,False,False
4,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# Split our preprocessed data into our features and target arrays
target_column = 'IS_SUCCESSFUL'
X = application_df.drop(columns=[target_column])
y = application_df[target_column]

# Display the shapes of the features and target arrays
print(X.shape)
print(y.shape)

(34299, 29)
(34299,)


In [ ]:
class_counts = application_df['IS_SUCCESSFUL'].value_counts()
print(class_counts)

IS_SUCCESSFUL
1    18261
0    16038
Name: count, dtype: int64


In [ ]:

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing sets
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(27439, 29)
(6860, 29)
(27439,)
(6860,)


In [ ]:
# Scale the ASK_AMT column (if it exists in your DataFrame)
if 'ASK_AMT' in X.columns:
    scaler = StandardScaler()
    X_train['ASK_AMT'] = scaler.fit_transform(X_train[['ASK_AMT']])
    X_test['ASK_AMT'] = scaler.transform(X_test[['ASK_AMT']])


## Compile, Train and Evaluate the Model

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assuming you have your DataFrame 'application_df' and target variable 'IS_SUCCESSFUL'
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

# One-hot encode categorical variables
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Determine the number of input features
total_input_features = X_train_scaled.shape[1]
print("Total number of input features:", total_input_features)



Total number of input features: 45


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Define the model
hidden_nodes_layer1 = 256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64

nn = tf.keras.models.Sequential()

# First hidden layer with Batch Normalization
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=X_train_scaled.shape[1], activation="relu"))
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))

# Second hidden layer with Batch Normalization
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))

# Third hidden layer with Batch Normalization
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Dropout(0.2))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Summary of the model
nn.summary()

# Custom checkpoint callback
class CustomModelCheckpoint(Callback):
    def __init__(self, save_freq=5):
        super(CustomModelCheckpoint, self).__init__()
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            self.model.save_weights(f'model_weights_epoch_{epoch + 1}.weights.h5')
            print(f'Saved weights at epoch {epoch + 1}')  # Print confirmation

# Define the number of epochs
num_epochs = 100  # Set this to the desired number of training epochs

# Create an instance of the custom checkpoint
custom_checkpoint = CustomModelCheckpoint(save_freq=5)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Learning rate reduction on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Train the model with the checkpoint and other callbacks
nn.fit(X_train_scaled, y_train, epochs=num_epochs, batch_size=32,
       validation_split=0.2,  # Use a validation split to monitor performance
       callbacks=[custom_checkpoint, early_stopping, reduce_lr])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 256)                 │          11,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 54,785 (214.00 KB)

 Trainable params: 53,889 (210.50 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.6712 - loss: 0.6539 - val_accuracy: 0.7365 - val_loss: 0.5532 - learning_rate: 0.0010
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7102 - loss: 0.5797 - val_accuracy: 0.7376 - val_loss: 0.5485 - learning_rate: 0.0010
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7154 - loss: 0.5723 - val_accuracy: 0.7374 - val_loss: 0.5501 - learning_rate: 0.0010
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7205 - loss: 0.5665 - val_accuracy: 0.7403 - val_loss: 0.5493 - learning_rate: 0.0010
Epoch 5/100
685/686 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7288 - loss: 0.5566Saved weights at epoch 5
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7288 - loss: 0.5567 - val_accuracy: 0.7409 - val_loss: 0.5483 - learning_rate: 0.0010
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7294 - loss: 0.5563 - val_accuracy: 0.7396 - val_loss: 0.5478 - learn

In [ ]:
# @title Default title text
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7283 - loss: 0.5524
Loss: 0.5523567199707031, Accuracy: 0.7282798886299133


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_v4.h5")